In [1]:
from qiskit import *
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
provider = IBMQ.get_provider(group='deployed')

In [5]:
from qat.interop.qiskit import qlm_to_qiskit
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
qiskit_circuit = qlm_to_qiskit(qlm_circuit)
print(qiskit_circuit)

      ┌───┐     ┌─┐   
q1_0: ┤ H ├──■──┤M├───
      └───┘┌─┴─┐└╥┘┌─┐
q1_1: ─────┤ X ├─╫─┤M├
           └───┘ ║ └╥┘
c1: 2/═══════════╩══╩═
                 0  1 


In [6]:


from qat.interop.qiskit import BackendToQPU
from qiskit import Aer
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

# Runtime using ibmq_qasm_simulator
backend = provider.backend.ibmq_qasm_simulator
qpu = BackendToQPU(backend)


result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

SamplerResult(quasi_dists=[{'11': 0.5009765625, '00': 0.4990234375}], metadata=[{'header_metadata': None, 'shots': 1024}])
State: |11>	 probability: 0.5009765625
State: |00>	 probability: 0.4990234375


In [8]:
from qat.interop.qiskit import AsyncBackendToQPU
from qiskit import Aer
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *
import time
nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

# Building a default AsyncBackendToQPU that's going to use qiskit's "qasm_simulator" if it couldn't connect
# to IBMQ using a token
qpu = AsyncBackendToQPU(backend)

async_job = qpu.submit(qlm_job)

print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
time.sleep(0.01)
print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
time.sleep(0.2)
print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
while async_job.result() is None:
    time.sleep(5)
result = async_job.result()
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))


ID: 626bf0c9171ee87d3a255f0b	 status : VALIDATING
ID: 626bf0c9171ee87d3a255f0b	 status : RUNNING
ID: 626bf0c9171ee87d3a255f0b	 status : RUNNING


KeyError: 0

In [ ]:
from qiskit import *
from qat.interop.qiskit import qiskit_to_qlm
from qiskit.circuit import Parameter

# Qiskit to QLM
qiskit_circuit = QuantumCircuit(2)
theta = Parameter("theta")
gamma = 3.14 - 2 * theta
qiskit_circuit.rx(theta, 0)
qiskit_circuit.rx(gamma, 1)

qlm_circuit = qiskit_to_qlm(qiskit_circuit)



In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(token=backend._api_client._credentials.token,
                              channel='ibm_quantum')

In [ ]:
backend = service.backend('ibmq_qasm_simulator')

In [ ]:
backend._api_client.account_api.session.base_url

In [ ]:
from qiskit import *
IBMQ.load_account()

In [ ]:
provider = IBMQ.get_provider(project='internal-test')
backend2 = provider.backend.ibmq_qasm_simulator
backend2.name

In [6]:
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit_ibm_runtime import QiskitRuntimeService

program_inputs = {
    'circuits': ReferenceCircuits.bell(),
     'circuit_indices': [0],
}
options = {'backend_name': backend.name()}
job = service.run(
    program_id="sampler",
    options=options,
    inputs=program_inputs)
print(f"job ID: {job.job_id}")
result = job.result()
result

NameError: name 'service' is not defined

In [ ]:
backend = service.backends()[0]
backend

In [7]:
from qiskit_ibm_runtime import Sampler
with Sampler(circuits=[ReferenceCircuits.bell()]*3, service=service, 
             options={ "backend": backend.name}) as sampler:
    result = sampler(circuit_indices=[0, 1, 2], transpile_config={'optimization_level': 3}, shots=1000)
    print(result)

NameError: name 'service' is not defined

In [ ]:
from qiskit.providers.ibmq import IBMQBackend
from qiskit_ibm_runtime import IBMBackend as RuntimeBackend

In [ ]:
result

In [ ]:
result.quasi_dists

In [ ]:
result.metadata[0]['shots']